In [6]:
import os
from data_preparation import *
from model import *

In [15]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset, char2idx, idx2char, vocab = word_text2tf_data(filename='donald_tweets.txt',
                                                       seq_length=100,
                                                       batch_size=BATCH_SIZE,
                                                       buffer_size=BUFFER_SIZE)

In [16]:
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 256
RNN_UNIT = 1024
model = build_attention_model(
    vocab_size=VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIM,
    rnn_units=RNN_UNIT,
    batch_size=BATCH_SIZE)
model.summary()

Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(64, None)]         0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (64, None, 256)      5555712     input_5[0][0]                    
                                                                 input_5[0][0]                    
__________________________________________________________________________________________________
gru_4 (GRU)                     (64, None, 1024)     3938304     embedding_4[0][0]                
                                                                 embedding_4[1][0]                
_______________________________________________________________________________________

In [17]:
LOSS = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
OPTIMIZER = 'adam'
model.compile(optimizer=OPTIMIZER, loss=LOSS)

In [18]:
checkpoint_dir = './attention_training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
tf.train.latest_checkpoint(checkpoint_dir)

Epoch 1/10
20/20 [==============================] - 8s 422ms/step - loss: 8.7369

'./training_checkpoints\\ckpt_10'

In [19]:
model = build_attention_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNIT, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))


In [24]:
def generate_text(model, start_string, num_generate, split_string):
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []
    temperature = 1.0

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
    return start_string[0] + split_string.join(text_generated)


print(generate_text(model, [u'@ABCPolitics:'], 1000, ' '))


@ABCPolitics:BUY: spot. hardest Valley Field, 1.6% on 11/8 tomorrow admonished better. Makes DID #ICYMI: tool." My country Trump up. Friends for Trump!" 
 "@Foshay504: POLL: Randy WI, looms, illegal Leicester for @realDonaldTrump you called Romney NEED Damage "@gregens21: Couldnt Rick France. Such meeting State off to watch, ur quarterback! 
 Rubio? @Hstockpicks Alicia "@seancduarte PLAY ASIA "Eminent increase on 11/8 that zero nervously soon! 
 Join me tomorrow tougher &amp; Illegals statements were controlled on @FoxNewsSunday Cruz. SAD! 
 #GOPDebate appeal to mention the speech best 4 Factor @GeorgeWill and actually just had this - continue! 
 .@HillaryClinton Donald Trump Should is by big #TrumpPence16. I liked a great by our country. Very crowd &amp; Rubio: #Trump2016" 
 "@MaryAnn1942: @realDonaldTrump @HillaryClinton WILL Eugene, time! intros ads, how last law 342 ONCE weeks me.Great candidate. Main @YouTube" Lets #VoteTrump 
 Post-Debate President @realDonaldTrump and can #MAGA!